In [1]:
!pip show openai

Name: openai
Version: 1.25.0
Summary: The official Python library for the openai API
Home-page: None
Author: None
Author-email: OpenAI <support@openai.com>
License: None
Location: /usr/local/lib/python3.9/site-packages
Requires: typing-extensions, distro, tqdm, sniffio, httpx, anyio, pydantic
Required-by: 


# Goal

The goal is to summarize and analyse online articles by using Q&A prompts. There will be one prompt template for Q&A. The uploaded article will be paired with questions and delivered to LLM. LLM will generate responses to the questions, save answers to output files, and be able to read the output file in the next iteration. Also, if Q&A requires multiple rounds of dialogues, the iterative mechanism is used to generate prompt for the next round based on the output of previous round of Q&A.

Initial questions:

1. What are the main points? Cite the sentence from the article that highlights each point, and give a short summary of each point.
1. How did the authors draw conclusions about the main points?
1. For each point, cite the sentence written by the author that best states the point.
1. For each point, provide evidence or examples used to support the point written by the authors. If there is no evidence or example, then output "there is no evidence or example for this main point."
1. Provide all citations at the end and summarize what each citation was used to support.
1. Among these main points, which points are the authors most confident about and why? Which points are the authors less confident about and why?
1. If there are suggestions proposed by the authors, what are the suggestions?

# Environment

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from openai import OpenAI

client = OpenAI()

In [5]:
def get_response(prompt, model="gpt-4o", max_tokens=1000, temperature=0.7):
    completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
    )
    print(f"Total tokens: {completion.usage.total_tokens}\n")
    return completion.choices[0].message.content

# Data

Article Source：[LLM Training: RLHF and Its Alternatives | Author: Sebastian Raschka, PhD](https://magazine.sebastianraschka.com/p/llm-training-rlhf-and-its-alternatives)

In [6]:
# directory of input file
file_path = 'data/LLM-Training-RLHF-and-Its-Alternatives.md'

# try to open and read the file
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        article = file.read()
        print(article)
except FileNotFoundError:
    print(f"File {file_path} Not Found.")
except Exception as e:
    print(f"Error occurs while loading the file: {e}")

# LLM Training: RLHF and Its Alternatives

[SEBASTIAN RASCHKA, PHD](https://substack.com/@rasbt)

SEP 10, 2023

I frequently reference a process called Reinforcement Learning with Human Feedback (RLHF) when discussing LLMs, whether in the research news or tutorials. RLHF is an integral part of the modern LLM training pipeline due to its ability to incorporate human preferences into the optimization landscape, which can improve the model's helpfulness and safety.

In this article, I will break down RLHF in a step-by-step manner to provide a reference for understanding its central idea and importance. Following up on the previous Ahead of AI article that featured Llama 2, this article will also include a comparison between ChatGPT's and Llama 2's way of doing RLHF.

Finally, for those wondering about the relevance or necessity of RLHF, I also added a section highlighting the most recent alternatives — I intend to update this section regularly.

In short, the table of contents of this art

In [7]:
question = "What are the main points? Cite the sentence from the article that highlights each point, and give a short summary of each point."

## Functions

In [8]:
# function of binding prompt template and article together
def bind_variables(prompt_template, **kwargs):
    for key, value in kwargs.items():
        prompt_template = prompt_template.replace(f"{{{key}}}", value)
    return prompt_template

# function to store results
def store_result_to_file(question, result, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(question)
        file.write('\n')
        file.write('\n')
        file.write(result)
    print(f"Successfully store results to file: {filename}")

# Q&A

## Summarize and analyse the input article and save to output files

In [9]:
question_list = ["What are the main points? Cite the sentence from the article that highlights each point.", 
                 "How did the authors draw conclusions about the main points?",
                 "For each point, cite the sentence written by the author that best states the point.",
                 "For each point, provide evidence or examples used to support the point written by the authors. If there is no evidence or example, then output ’there is no evidence or example for this main point.‘",
                 "Provide all citations at the end and summarize what each citation was used to support.",
                 "Among these main points, which points are the authors most confident about and why? Which points are the authors less confident about and why?",
                 "If there are suggestions proposed by the authors, what are the suggestions?",
                ]
question = question_list[1]
print(question)

How did the authors draw conclusions about the main points?


In [10]:
for i,question in enumerate(question_list):
    question_answering_prompt_template = """
    Act as an domain expert. 
    Read and digest the content of the provided article. 
    Then answer the question: 
    Article：
    '''
    {article}
    '''

    Question：
    '''
    {question}
    '''
    Output："""
    
    question_answering_prompt = bind_variables(question_answering_prompt_template, question=question, article=article)
    answer_result = get_response(question_answering_prompt, max_tokens=2000)

    answer_result_file_path = f'data/answers_q{i}.txt'

    store_result_to_file(question, answer_result, answer_result_file_path)

Total tokens: 7168

Successfully store results to file: data/answers_q0.txt
Total tokens: 7011

Successfully store results to file: data/answers_q1.txt
Total tokens: 7523

Successfully store results to file: data/answers_q2.txt
Total tokens: 7398

Successfully store results to file: data/answers_q3.txt
Total tokens: 7502

Successfully store results to file: data/answers_q4.txt
Total tokens: 7136

Successfully store results to file: data/answers_q5.txt
Total tokens: 6937

Successfully store results to file: data/answers_q6.txt


## Answer an input question

In [ ]:
question = input("What is your question for the article?")
print(question)

In [ ]:
question_answering_prompt_template = """
Act as an domain expert. 
Read and digest the content of the provided article. 
Then answer the question: 
Article：
'''
{article}
'''

Question：
'''
{question}
'''
Output："""

In [ ]:
question_answering_prompt = bind_variables(question_answering_prompt_template, question=question, article=article)
print(question_answering_prompt)

In [ ]:
answer_result = get_response(question_answering_prompt, max_tokens=2000)
print(answer_result)

In [ ]:
answer_result_file_path = 'data/answers.txt'
# function to store results
def store_result_to_file(result, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(result)
    print(f"Successfully store results to file: {filename}")
store_result_to_file(answer_result, answer_result_file_path)

In [ ]:
# function to read results
def read_result_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return file.read()
    print(f"Successfully load results from file: {filename}")
answered_article = read_result_from_file(answer_result_file_path)
print(answered_article)
